# Introduction to Random Forest Classifier

This notebook will use the Iris flower dataset from sklearn to introduce classification with Random Forest.

Key distinctions between decision tree and random forest are explored. 

Feature importance and partial dependency plots for random forest are generated post-training to facilitate model visibility. 

# Overview

- Set up train-test data
- Review decision trees
- Visualize a decision tree
- Introduction to random forest
- Train and predict with a random forest
- Visualize feature importances
- Create partial dependency plots for random forest
- Knowledge check and questions
- Additional EDA & Visualizations

## Prerequisite setup and knowledge
- Correct Python environment
- Train-test split
- Classification metrics
- Decision Trees
- Measures of node impurity (Shannon Entropy and Gini Index)

# Learning Objectives

1. Apply a random forest classifier to a dataset
1. Visualize feature importances from a trained random forest model

# Set up data set 

In [ ]:
import pandas as pd
from sklearn import datasets
import seaborn as sns
import skimpy

In [ ]:
data = datasets.load_iris()
df = pd.DataFrame(data.data, columns=data.feature_names)
df['target'] = data.target
df['label'] = df['target'].map({0: 'setosa', 1: 'versicolor', 2: 'virginica'})
df.head()

In [ ]:
df.shape

In [ ]:
df.label.value_counts()

In [ ]:
df.describe()

## Train-Test Split

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    df[data.feature_names], 
    df['label'], 
    random_state=42,
    stratify=df['label']
)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

# Decision Tree Review

We will first build a decision tree model to review their structure before moving on to random forest classification

In [ ]:
from matplotlib import pyplot as plt
from sklearn.tree import DecisionTreeClassifier 
from sklearn import tree

In [ ]:
dt = DecisionTreeClassifier(random_state=42)
dt.fit(X_train, y_train)

In [ ]:
fig = plt.figure(figsize=(25,20))
_ = tree.plot_tree(dt, 
                   feature_names=data.feature_names,  
                   class_names=data.target_names,
                   filled=True)

In [ ]:
predicted_labels = dt.predict(X_test)

In [ ]:
from sklearn.metrics import (
    accuracy_score,
    recall_score,
    precision_score,
    f1_score
)

def print_metrics(y_test, y_pred):
    """function to display model metrics in a human readable fashion
    y_test: Array of true y values
    y_pred: Arrah of predicted y values
    """
    scores = [accuracy_score, recall_score, precision_score, f1_score]
    s_labels = ['Accuracy', 'Recall', 'Precision', 'F1']
    for score, s_label in zip(scores, s_labels):
        if s_label == 'Accuracy':
            print(f"{s_label}: {score(y_test, y_pred): .2f}")
        else:
            print(f"{s_label}: {score(y_test, y_pred, average='weighted'):.2f}")

In [ ]:
print_metrics(y_test, predicted_labels)

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

cm_disp = ConfusionMatrixDisplay.from_predictions(y_true=y_test,y_pred=predicted_labels, cmap='Blues')

# Decision Tree Knowledge
1. Are decision trees deterministic?
    - Yes they are deterministic. The best split will be found at each iterative step and will be used. 
1. How are decision trees split determined?
    - Information gain or entropy reduction
1. Are decision trees parametric? 
    - No, they are not parametric. Splits may differ in direction based on values.
1. Decision trees often have high variance, why might that be?
    - May split on wrong features or overfit to data.

# From tree to forest

1. How might multiple decision trees be leveraged to reduce variance?
    1. Create multiple classifiers and average the results. Multiple weak learners can ofter produce a strong learner.
1. Would multiple deterministic decision trees be useful?
    1. Only if they were NOT deterministic.
1. How could they not be deteministic?
    1. Bootstrapping data and limiting features at each step

# Random Forest
- An ensemble method that combines many decision trees which have been given different subsets of the data and features to create a strong learner 
- Decision made on majority vote
- Reduces variance and creates a non-deterministic model
- Generally use a large number of bushy trees
- Can get excellent performance with minimum tuning


# Random Forest Example

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=10, random_state=1)
rf.fit(X_train, y_train)
rf_preds = rf.predict(X_test)

In [ ]:
cm_disp = ConfusionMatrixDisplay.from_predictions(y_true=y_test, y_pred=rf_preds, cmap='Blues')

In [ ]:
print_metrics(y_test, rf_preds)

# Visualizing trees in the forest

In [ ]:
def plot_tree(est_num=0):
    fn=data.feature_names
    cn=data.target_names
    tree.plot_tree(rf.estimators_[est_num],
                   feature_names = fn, 
                   class_names = cn,
                   filled = True)

In [ ]:
plot_tree(est_num=0)

In [ ]:
plot_tree(est_num=1)

Focusing only on the first split in the two trees above, we can see differences in the splits used to build the forest.

The first one split on sepal lenth <= 5.35 and the second on petal length <=2.45. The depth of the trees also varies. This is due to the randomness induced in the trees with bootstrapping and feature selection. 


# Random Forest Feature Importances

In [ ]:
ft_import = pd.DataFrame()
ft_import['Features'] = data.feature_names
ft_import['Importance'] = rf.feature_importances_
ft_import

In [ ]:
# All importances sum to 1
ft_import.Importance.sum()

In [ ]:
ft_import.plot.bar(x='Features', y='Importance', rot=30)

# Partial Dependency Plots

Shows the marginal effect of a feature on predictions.

Shows effect of predictions when all observations have a feature set to a particular value.


In [ ]:
from sklearn.inspection import PartialDependenceDisplay

pd_plot = PartialDependenceDisplay.from_estimator(rf, X_train, ['petal length (cm)'], target='setosa')

In [ ]:
for target in data.target_names:
    print('Target is: ', target)
    PartialDependenceDisplay.from_estimator(rf, X_train, data.feature_names, target=target)

# Advantages of Random Forests

1. Ensemble model (Wisdom of the Crowd)
1. Good out-of-box performance
1. Multiple trees can be trained at once 

# Cons of Random Forests
1. Expensive to train
2. Can produce very large model files

# Model Comparison and evaluation

1. Which model performed better?
2. Which feature had the most influence on the random forest model?


# Knowledge Check
1. A random forest is the same as combining many decision trees?
1. Name two ways in which random forests are made non-deterministic. 
1. Random forest classifiers are parametric?
1. Name two visuals to help intrepret random forest models.

# Review Objectives 
1. Apply a random forest classifier to a dataset.
1. Visualize feature importances from a trained random forest model.

# Next Steps
1. Tune the random forest classifier
    1. Number of estimators
    1. Criterion: default is gini, can also try entropy
    1. Max depth, min samples
    1. Number of features
1. Improve visuals
   1. Add title to partial dependency plots
   1. Create interactive visuals with Plotly for scatter and bar plots below
   1. Edit layout so all axis labels are fully visible

# Full Day Activities
1. Code random forest from scratch
1. Code partial dependecy plot function from scratch

# Additional EDA & Visualizations

In [ ]:
skimpy.skim(df)

In [ ]:
features = df.columns[:4]
features

In [ ]:
sns.scatterplot(data=df, x='sepal length (cm)', y='sepal width (cm)', hue='label')

In [ ]:
sns.scatterplot(data=df, x='petal length (cm)', y='petal width (cm)', hue='label')